In [1]:
import pandas as pd
from math import log
from statistics import mean,variance
from math import exp,sqrt,pi

In [2]:
def calc_probability(con_col_name='',con_val='',target_col_name='',target_val='',training_data='',smooth_factor_dict={},mean_n_variance_dict={},attr_dict={}):
    probability=0
    if con_col_name=='':
        comp=training_data[target_col_name]==target_val
        count_of_cond_tar=sum(comp)
        probability=count_of_cond_tar/len(training_data)
        
    elif attr_dict[con_col_name]=='CAT':
        comp=training_data[target_col_name]==target_val
        count_of_cond_tar=sum(comp)
        count_of_cond=sum(training_data[comp][con_col_name]==con_val)
        probability=(count_of_cond+1)/(count_of_cond_tar+smooth_factor_dict[con_col_name])
    
    elif attr_dict[con_col_name]=='CON':
        mea=mean_n_variance_dict[con_col_name][target_val][0]
        var=mean_n_variance_dict[con_col_name][target_val][1]
        probability=exp(-1*((target_val-mea)**2)/(2*var))/sqrt(2*pi*var)
        
        
        
    return probability     

In [3]:
def calc_smoothing_factor(training_data,training_cols,attr_dict):
    smooth_factor_dict={}
    for col in training_cols:
        if attr_dict[col]=='CAT':
            smooth_factor_dict[col]=training_data[col].nunique()
        
    return smooth_factor_dict

In [4]:
def calc_mean_n_variance(training_data,training_cols,attr_dict,all_classes,target_col_name):
    mean_n_variance_dict={}
    for col in training_cols:
        if attr_dict[col]=='CON':
            all_classes_dict={}
            for e_class in all_classes:
                mea=mean(list(training_data[training_data[target_col_name]==e_class][col]))
                var=variance(list(training_data[training_data[target_col_name]==e_class][col]))
                all_classes_dict[e_class]=[mea,var]
            mean_n_variance_dict[col]=all_classes_dict
        
    return mean_n_variance_dict

In [5]:
#Reading the training and testing csv files downloaded from kaggle
df=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')
df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [6]:
#Setting all necessary attributes for training and target
training_cols=['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'] #PassengerId,Name,Ticket,Cabin was not included
target_col='Survived'
all_cols=[x for x in training_cols]
all_cols.append(target_col)
attr_dict={'PassengerId':'CAT', 
           'Pclass':'CAT', 
           'Name':'CAT', 
           'Sex' :'CAT',
           'Age':'CON',
           'SibSp':'CAT', 
           'Parch':'CAT', 
           'Ticket' :'CAT',
           'Fare':'CON',
           'Cabin':'CAT',
           'Embarked':'CAT'}

In [7]:
#Cleaning the training data
df_clean=df[all_cols]
df_clean=df_clean[~df_clean.isna().any(axis=1)]

In [8]:
#Calculating all the necessary parameters from Training Data
all_classes=df_clean[target_col].unique()
df_predicted=pd.DataFrame()
smooth_factor_dict=calc_smoothing_factor(df_clean,training_cols,attr_dict)
mean_n_variance_dict=calc_mean_n_variance(df_clean,training_cols,attr_dict,all_classes,target_col)

#Predicting for all the rows in test_data i.e. df_test data frame
for (index,row) in df_test.iterrows():
    probability_dict={}
    for pred_class in all_classes:        
        sum_of_logs=0
        for col in training_cols:
            sum_of_logs+=log(calc_probability(col,row[col],target_col,pred_class,df_clean,smooth_factor_dict,mean_n_variance_dict,attr_dict),2)
        sum_of_logs+=log(calc_probability(target_col_name=target_col,target_val=pred_class,training_data=df_clean),2)
        probability_dict[pred_class]=sum_of_logs
    row_dict=row.to_dict()
    row_dict[target_col]=max(probability_dict.keys(),key=lambda x:probability_dict[x])
    df_predicted=df_predicted.append(row_dict,ignore_index=True)

df_predicted    
# print(max(probability_dict.keys(),key=lambda x:probability_dict[x]))
# print(probability_dict)

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
0,34.5,NaN,Q,7.8292,"Kelly, Mr. James",0.0,892.0,3.0,male,0.0,0.0,330911
1,47.0,NaN,S,7.0000,"Wilkes, Mrs. James (Ellen Needs)",0.0,893.0,3.0,female,1.0,1.0,363272
2,62.0,NaN,Q,9.6875,"Myles, Mr. Thomas Francis",0.0,894.0,2.0,male,0.0,0.0,240276
3,27.0,NaN,S,8.6625,"Wirz, Mr. Albert",0.0,895.0,3.0,male,0.0,0.0,315154
4,22.0,NaN,S,12.2875,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1.0,896.0,3.0,female,1.0,1.0,3101298
...,...,...,...,...,...,...,...,...,...,...,...,...
413,NaN,NaN,S,8.0500,"Spector, Mr. Woolf",0.0,1305.0,3.0,male,0.0,0.0,A.5. 3236
414,39.0,C105,C,108.9000,"Oliva y Ocana, Dona. Fermina",0.0,1306.0,1.0,female,0.0,1.0,PC 17758
415,38.5,NaN,S,7.2500,"Saether, Mr. Simon Sivertsen",0.0,1307.0,3.0,male,0.0,0.0,SOTON/O.Q. 3101262
416,NaN,NaN,S,8.0500,"Ware, Mr. Frederick",0.0,1308.0,3.0,male,0.0,0.0,359309


In [9]:
df_actual=pd.read_csv('gender_submission.csv')
df_actual

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [10]:
acc=sum(df_actual.sort_values(by='PassengerId').Survived==
        df_predicted.sort_values(by='PassengerId').Survived)*100/len(df_actual)
print('Test Data Accuracy: {}%'.format(acc))

Test Data Accuracy: 84.92822966507177%


In [11]:
mean_n_variance_dict

{'Age': {0: [30.62617924528302, 200.84869836968642],
  1: [28.19329861111111, 220.79422845407473]},
 'Fare': {0: [22.965456367924528, 989.0285899001484],
  1: [51.647671875, 4993.471436896245]}}